In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q "drive/MyDrive/data/cropped.zip"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import pathlib
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing import image

In [ ]:
data_dir = pathlib.Path('drive/MyDrive/data/train_phone')

In [ ]:
# reduce batch size
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(128, 128),
    batch_size=20,
    interpolation='nearest'
)

In [ ]:
# reduce batch size
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(128, 128),
    batch_size=20,
    interpolation='nearest'
)

In [ ]:
class_names = train_ds.class_names
plt.figure(figsize=(10, 10))
for images, labels in tqdm(train_ds.take(1)):
    for i in tqdm(range(9)):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
num_classes = 40
model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(128, 128, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='Adam'
  , loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  , metrics=['accuracy'])

In [ ]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='drive/MyDrive/data/phone_{epoch:02d}.keras',
    save_weights_only=False,
    period=1
)

In [ ]:
epochs=10
history=model.fit(train_ds, validation_data=validation_ds, epochs=epochs, callbacks=[checkpoint_callback])

In [ ]:
model.save('drive/MyDrive/data/phone.keras')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
loss, accuracy = model.evaluate(validation_ds)
print('Test accuracy :', accuracy)

In [ ]:
print("Model input shape:", model.input_shape)
print("Model output shape:", model.output_shape)
print("Number of layers:", len(model.layers))
print("Layer names:", [layer.name for layer in model.layers])


In [ ]:
model = keras.models.load_model('drive/MyDrive/data/phone_05.h5')

In [ ]:
class_names = train_ds.class_names
image_path = "cropped/010890.jpg"  # Replace with the actual path
img = image.load_img(image_path, target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

In [ ]:
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions[0])
predicted_class = class_names[predicted_class_index]

print("Predicted class:", predicted_class)

In [ ]:
top_3_indices = np.argsort(predictions[0])[::-1][:3]
top_3_classes = [class_names[idx] for idx in top_3_indices]
top_3_probabilities = predictions[0][top_3_indices]

In [ ]:
print("Top 3 predicted classes:")
for class_name, probability in zip(top_3_classes, top_3_probabilities):
    print(f"{class_name}: {probability:.2f}")

In [ ]:
img = image.load_img(image_path, target_size=(128, 128))
img_array = image.img_to_array(img)

df = pd.read_csv('drive/MyDrive/data/data.csv')
image_name = f"cropped/{image_path}"
actual_class = df.loc[df['image'] == image_name, 'phone'].values[0]
print(f"Actual class for {image_name}: {actual_class}")


plt.imshow(img_array.astype("uint8"))
plt.title(f"Actual class: {actual_class}")
plt.axis("off")
plt.show()